In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install pyannote.audio==3.1.1 yt-dlp faster-whisper --quiet

# !apt-get -qq install -y ffmpeg

# # !pip install --upgrade --force-reinstall torch==2.1.2 torchaudio==2.1.2 torchvision==0.16.2 --index-url https://download.pytorch.org/whl/cu118

# !pip install numpy==1.26.4 --quiet

# # !pip install -U --quiet transformers==4.40.1

In [ ]:
# !pip uninstall -y nvidia-cudnn-cu12 nvidia-cudnn-cu11

# !pip install --quiet --upgrade --index-url https://download.pytorch.org.whl/cu118 torch==2.2.2+cu118 torchaudio==2.2.2+cu118

# !pip install --no-deps pyannote.audio==3.1.1

In [ ]:
# !pip install --quiet numpy==1.26.4

# !pip install --quiet faster_whisper pyannote.audio==3.1.1 yt-dlp

# !apt-get -qq install -y ffmpeg

In [ ]:
# Langkah 1: Instal PyTorch versi spesifik yang stabil.
!pip install --quiet torch==2.1.2 torchaudio==2.1.2 torchvision==0.16.2 --index-url https://download.pytorch.org/whl/cu118

# Langkah 2: Instal semua library lain dengan versi yang sudah divalidasi kompatibel.
# KUNCI PERBAIKAN: menaikkan versi huggingface-hub ke 0.22.2
!pip install --quiet \
    numpy==1.26.4 \
    pyannote.audio==3.1.1 \
    faster-whisper \
    yt-dlp \
    datasets \
    sentencepiece \
    accelerate \
    evaluate \
    rouge_score \
    transformers==4.40.1 \
    tokenizers==0.19.1 \
    huggingface-hub==0.22.2

# Langkah 3: Instal ffmpeg.
!apt-get -qq install -y ffmpeg

print("✅ Semua dependensi berhasil diinstal dengan versi yang kompatibel.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 480.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 79.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 109.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.7/208.7 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 125.0 MB/s eta 0:00:

In [ ]:
import os
import torch
import gc
from datetime import timedelta
from huggingface_hub import login
from faster_whisper import WhisperModel
from pyannote.audio import Pipeline
from transformers import pipeline as hf_pipeline
from google.colab import files

HUGGINGFACE_TOKEN = "input token hugginface disini"
device = "cuda" if torch.cuda.is_available() else "cpu"

# Path untuk file-file sementara yang akan dibuat oleh skrip
TEMP_AUDIO_PATH = "temp_processing_audio.wav"
OUTPUT_VIDEO_PATH = "output_with_subtitles.mp4"
SUBTITLE_PATH = "subtitles.srt"
DEFAULT_INPUT_VIDEO_NAME = "input_video.mp4"

# Login ke Hugging Face
login(token=HUGGINGFACE_TOKEN, add_to_git_credential=False)

/usr/local/lib/python3.11/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
def preprocess_media(input_path, audio_output_path):
    """Mengekstrak audio dari file video/audio dan mengonversinya ke format WAV 16kHz mono."""
    print(f"--- Memproses file media: {input_path} ---")
    # Perintah FFmpeg:
    # -y: Timpa file output jika sudah ada
    # -i: File input
    # -vn: Hapus stream video (hanya ambil audio)
    # -acodec pcm_s16le: Format audio WAV standar
    # -ar 16000: Sample rate 16kHz (standar untuk model speech-to-text)
    # -ac 1: Channel audio mono
    command = f'ffmpeg -y -i "{input_path}" -vn -acodec pcm_s16le -ar 16000 -ac 1 "{audio_output_path}"'
    os.system(command)
    print(f"--- Audio berhasil diekstrak dan disimpan di {audio_output_path} ---")
    return audio_output_path

def transcribe_audio(audio_path):
    """Mengubah audio menjadi teks menggunakan Faster-Whisper."""
    transcription_segments = []
    try:
        print("\n--- Memulai Transkripsi ---")
        # Model 'medium' adalah keseimbangan yang baik antara kecepatan dan akurasi.
        # compute_type="float16" mempercepat proses di GPU.
        model = WhisperModel("medium", device=device, compute_type="default")
        segments, info = model.transcribe(audio_path, beam_size=5)
        transcription_segments = list(segments)
        print(f"-> Bahasa terdeteksi: {info.language} (Probabilitas: {info.language_probability:.2f})")
    except Exception as e:
        print(f"Error saat transkripsi: {e}")
    finally:
        # Membersihkan memori GPU setelah selesai
        if 'model' in locals(): del model
        gc.collect()
        torch.cuda.empty_cache()
    print("--- Transkripsi Selesai ---")
    return transcription_segments

def diarize_speakers(audio_path):
    """Mengidentifikasi giliran bicara setiap pembicara menggunakan Pyannote."""
    speaker_turns = []
    try:
        print("\n--- Memulai Diarisasi Pembicara ---")
        diarization_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1")
        diarization_pipeline.to(torch.device(device))
        diarization = diarization_pipeline(audio_path)

        # Mengambil hasil diarisasi: waktu mulai, waktu selesai, dan label pembicara
        for turn, _, speaker in diarization.itertracks(yield_label=True):
            speaker_turns.append({'start': turn.start, 'end': turn.end, 'speaker': speaker})
        print(f"-> Diarisasi selesai. Ditemukan {len(diarization.labels())} pembicara.")
    except Exception as e:
        print(f"Error saat diarisasi: {e}")
    finally:
        if 'diarization_pipeline' in locals(): del diarization_pipeline
        gc.collect()
        torch.cuda.empty_cache()
    print("--- Diarisasi Selesai ---")
    return speaker_turns

def format_time_srt(seconds):
    """Mengubah detik menjadi format waktu untuk file subtitle (.srt)."""
    td = timedelta(seconds=seconds)
    minutes, seconds = divmod(td.seconds, 60)
    hours, minutes = divmod(minutes, 60)
    milliseconds = td.microseconds // 1000
    return f"{hours:02d}:{minutes:02d}:{seconds:02d},{milliseconds:03d}"

def generate_final_transcript_and_srt(transcription_segments, speaker_turns, srt_path):
    """Menggabungkan transkrip dan diarisasi, mencetak hasilnya, dan membuat file .srt."""
    full_transcript_text = ""
    srt_content = ""
    srt_index = 1

    print("\n--- Hasil Akhir Transkrip dengan Pembicara ---")

    def get_speaker_label(time, turns):
        for turn in turns:
            if turn['start'] <= time <= turn['end']:
                return turn['speaker']
        return "UNKNOWN"

    for segment in transcription_segments:
        # Tentukan pembicara berdasarkan titik tengah segmen waktu
        mid_time = segment.start + (segment.end - segment.start) / 2
        speaker_label = get_speaker_label(mid_time, speaker_turns)

        # Teks yang akan ditampilkan di console
        line = f"[{speaker_label}] ({format_time_srt(segment.start)} --> {format_time_srt(segment.end)}): {segment.text.strip()}"
        print(line)

        # Gabungkan teks untuk analisis selanjutnya (ringkasan & Q&A)
        full_transcript_text += segment.text.strip() + " "

        # Buat konten untuk file .srt
        start_time_str = format_time_srt(segment.start)
        end_time_str = format_time_srt(segment.end)
        srt_content += f"{srt_index}\n"
        srt_content += f"{start_time_str} --> {end_time_str}\n"
        srt_content += f"[{speaker_label}] {segment.text.strip()}\n\n"
        srt_index += 1

    # Tulis konten ke file .srt
    with open(srt_path, 'w', encoding='utf-8') as f:
        f.write(srt_content)
    print(f"\n--- File subtitle telah disimpan di: {srt_path} ---")

    return full_transcript_text.strip()

def burn_subtitles_into_video(video_path, subtitle_path, output_video_path):
    """Menggabungkan file subtitle ke dalam video untuk membuat 'hardsub'."""
    print(f"\n--- Menambahkan subtitle ke video: {video_path} ---")
    # Perintah FFmpeg untuk "membakar" subtitle ke video
    command = f'ffmpeg -y -i "{video_path}" -vf "subtitles={subtitle_path}" "{output_video_path}"'
    os.system(command)
    print(f"--- Video dengan subtitle berhasil dibuat di: {output_video_path} ---")
    print("Anda dapat mengunduhnya dari panel file di sebelah kiri.")

####################################
# FUNGSI LAMA SUDAH TIDAK DIPAKAI #
###################################
def perform_text_analysis(text):
    """Melakukan ringkasan dan tanya jawab pada transkrip lengkap."""
    print("\n--- Memulai Analisis Teks ---")

    # Summarization
    try:
        summarizer = hf_pipeline("summarization", model="facebook/bart-large-cnn", device=device)
        # Batasi input untuk model ringkasan agar tidak terlalu panjang
        summary_input = text[:4096]
        summary = summarizer(summary_input, max_length=150, min_length=50, do_sample=False)
        print("\n**Ringkasan Otomatis:**")
        print(summary[0]['summary_text'])
    except Exception as e:
        print(f"Error saat meringkas: {e}")
    finally:
        if 'summarizer' in locals(): del summarizer
        gc.collect(); torch.cuda.empty_cache()

    # Question Answering
    try:
        qa_pipeline = hf_pipeline("question-answering", model="deepset/roberta-base-squad2", device=device)
        questions = [
            "What is the main topic of the conversation?",
            "Who are the speakers mentioned by name?",
            "What is the key advice or conclusion?"
        ]
        print("\n**Tanya Jawab Otomatis:**")
        for q in questions:
            result = qa_pipeline(question=q, context=text)
            answer = result['answer'] if result['score'] > 0.1 else "Tidak ditemukan jawaban yang relevan."
            print(f"\nQ: {q}")
            print(f"A: {answer}")
    except Exception as e:
        print(f"Error saat tanya jawab: {e}")
    finally:
        if 'qa_pipeline' in locals(): del qa_pipeline
        gc.collect(); torch.cuda.empty_cache()

In [ ]:
# --- Membersihkan file dari run sebelumnya ---
for path in [TEMP_AUDIO_PATH, SUBTITLE_PATH, OUTPUT_VIDEO_PATH, DEFAULT_INPUT_VIDEO_NAME]:
    if os.path.exists(path):
        os.remove(path)

# --- Menu Pilihan Input untuk Pengguna ---
# print("=====================================================")
# print("      SISTEM ANALISIS AUDIO/VIDEO CERDAS      ")
# print("=====================================================")
# print("Pilih metode input:")
# print("1: Gunakan link YouTube")
# print("2: Unggah file dari komputer (MP4, WAV, MP3, dll.)")
# choice = input("Masukkan pilihan Anda (1 atau 2): ")

# input_file_path = None
# is_video = False

# # --- Jalur 1: Proses Link YouTube ---
# if choice == '1':
#     youtube_url = input("Masukkan link YouTube: ")
#     if youtube_url:
#         print(f"\n--- Mengunduh video dari: {youtube_url} ---")
#         # Mengunduh video sebagai MP4 dengan nama file yang konsisten
#         os.system(f"yt-dlp -f 'best[ext=mp4]/best' -o '{DEFAULT_INPUT_VIDEO_NAME}' --quiet '{youtube_url}'")

#         if os.path.exists(DEFAULT_INPUT_VIDEO_NAME):
#             input_file_path = DEFAULT_INPUT_VIDEO_NAME
#             is_video = True
#             print("--- Video berhasil diunduh! ---")
#         else:
#             print("--- GAGAL mengunduh video. Pastikan link valid dan video tidak memiliki batasan umur/privat. ---")

# # --- Jalur 2: Proses Unggah File ---
# elif choice == '2':
#     print("\n--- Silakan unggah file Anda melalui tombol di bawah ---")
#     uploaded = files.upload()

#     if uploaded:
#         input_file_path = list(uploaded.keys())[0]
#         if input_file_path.lower().endswith(('.mp4', '.mov', '.avi', '.mkv')):
#             is_video = True
#         print(f"--- File '{input_file_path}' berhasil diunggah! ---")
#     else:
#         print("Tidak ada file yang diunggah. Proses dibatalkan.")

# # --- Pilihan tidak valid ---
# else:
#     print("Pilihan tidak valid. Harap jalankan sel lagi dan pilih 1 atau 2.")


# if input_file_path:
#     print("\n\n================= MEMULAI PROSES ANALISIS =================\n")

#     # 1. Ekstrak Audio dari file input
#     audio_path = preprocess_media(input_file_path, TEMP_AUDIO_PATH)

#     # 2. Transkripsi
#     transcription_segments = transcribe_audio(audio_path)

#     if transcription_segments:
#         # 3. Diarisasi
#         speaker_turns = diarize_speakers(audio_path)

#         if speaker_turns:
#             # 4. Gabungkan hasil & Buat file .srt
#             full_text = generate_final_transcript_and_srt(transcription_segments, speaker_turns, SUBTITLE_PATH)

#             # 5. Jika input adalah video, buat video dengan hardsub
#             if is_video:
#                 burn_subtitles_into_video(input_file_path, SUBTITLE_PATH, OUTPUT_VIDEO_PATH)

#             # 6. Lakukan Analisis Teks (Ringkasan & Q&A)
#             if full_text:
#                 perform_text_analysis(full_text)
#         else:
#             print("\nPROSES GAGAL: Diarisasi tidak berhasil. Pipeline dihentikan.")
#     else:
#         print("\nPROSES GAGAL: Transkripsi tidak menghasilkan output. Pipeline dihentikan.")

In [ ]:
# =========================================
#  GRADIO UI
# =========================================
import gradio as gr
import subprocess, tempfile, json, pathlib, shutil
from transformers import pipeline as hf_pipeline
import traceback, time
import html

# --- pipeline ringan utk ringkasan & QA ---
# summarizer   = hf_pipeline("summarization", model="facebook/bart-large-cnn",    device="cpu")
# qa_pipeline  = hf_pipeline("question-answering", model="deepset/roberta-base-squad2", device="cpu")

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Gunakan device yang tersedia (GPU jika ada)
device = "cuda" if torch.cuda.is_available() else "cpu"

# Path ke model fine-tuned Anda di Google Drive
load_path = "/content/drive/MyDrive/flan-t5-dialogsum-finetuned"
summary_model = None
summary_tokenizer = None

# Coba muat model kustom Anda
try:
    print(f"Loading custom model from: {load_path}")
    summary_tokenizer = AutoTokenizer.from_pretrained(load_path)
    summary_model = AutoModelForSeq2SeqLM.from_pretrained(load_path, trust_remote_code=True).to(device)
    print("✅ Successfully loaded fine-tuned FLAN-T5 summarization model.")
except Exception as e:
    print(f"❌ ERROR: Could not load the model from {load_path}. Error: {e}")
    print("   Summarization will be disabled.")

# Muat pipeline untuk Question-Answering (tidak berubah)
qa_pipeline  = hf_pipeline("question-answering", model="deepset/roberta-base-squad2", device=device)

def summarize_with_flan_t5(text, model, tokenizer):
    """
    Menghasilkan ringkasan menggunakan model FLAN-T5 yang telah di-fine-tune.
    """
    if model is None or tokenizer is None:
        return "Summarization model not loaded. Skipping."

    # 1. Tambahkan prefix yang diperlukan
    input_text = "summarize: " + text

    # 2. Tokenisasi input
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=1024).to(model.device)

    # 3. Generate ringkasan
    summary_ids = model.generate(**inputs, max_new_tokens=128)

    # 4. Decode output
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# --- util parse .srt → list dict ----------------------------------------------
import re
def srt_to_dialog_list(srt_path):
    dialog = []
    with open(srt_path, "r", encoding="utf-8") as f:
        blocks = re.split(r"\n\s*\n", f.read().strip())
    for blk in blocks:
        lines = blk.splitlines()
        if len(lines) < 3:               # nomor + waktu + text
            continue
        text = " ".join(lines[2:]).strip()
        m = re.match(r"\[([^\]]+)]\s*(.+)", text)
        speaker, content = ("UNKNOWN", text) if not m else m.groups()
        dialog.append({"speaker": speaker, "content": content})
    return dialog

# --- konversi ke format gr.Chatbot: list[tuple(str,str)] -----------------------
def dialog_to_chat_tuples(dialog):
    return [(d["speaker"], d["content"]) for d in dialog]


def dialog_to_html(dialog):
    """
    Ubah list [{"speaker":..,"content":..},…] → HTML chat bubble warna-warni.
    """
    palette = [
      "#0d47a1",  # navy-blue
      "#4a148c",  # dark-purple
      "#1b5e20",  # hunter-green
      "#880e4f",  # wine-red
      "#b71c1c",  # dark-crimson
      "#004d40",  # teal-dark
      "#3e2723",  # espresso-brown
      "#263238"   # slate-gray
    ]
    speaker_color = {}
    color_idx = 0
    lines = []

    for d in dialog:
        spk = d["speaker"]
        if spk not in speaker_color:
            speaker_color[spk] = palette[color_idx % len(palette)]
            color_idx += 1
        color = speaker_color[spk]
        text = html.escape(d["content"])
        lines.append(
            f"<div style='background:{color};border-radius:8px;padding:8px;"
            f"margin-bottom:6px;'><strong>{spk}:</strong> {text}</div>"
        )
    return "<div style='font-family:Arial, sans-serif;line-height:1.4;'>" + "\n".join(lines) + "</div>"

# --- core wrapper: jalankan pipeline lama --------------------------------------
import traceback, time
import gradio as gr

def run_full_pipeline(youtube_url, uploaded_file, progress=gr.Progress()):
    """
    Jalankan pipeline + debug.
    Semua exception ditangkap, dicetak stack-trace, dan dilempar ke UI sebagai gr.Error.
    """
    t0 = time.perf_counter()          # start timer

    def log_and_raise(step, err):
        """Cetak stack trace + lempar error ke Gradio."""
        print(f"\n[DEBUG] ERROR saat {step}: {err}")
        traceback.print_exc()         # full stack di console
        raise gr.Error(f"Gagal di tahap '{step}': {err}")

    # 0. Sumber input
    try:
        if youtube_url:
            progress(0, desc="Mengunduh video...")
            print(f"--- Mencoba mengunduh dari YouTube: {youtube_url} ---")

            # Perintah yt-dlp sebagai list untuk subprocess
            command = [
                "yt-dlp",
                "-f", "best[ext=mp4]/best",
                "-o", DEFAULT_INPUT_VIDEO_NAME,
                youtube_url
            ]

            # Menggunakan subprocess untuk menangkap output dan error
            # Ini akan memberi kita alasan pasti jika gagal
            result = subprocess.run(command, capture_output=True, text=True)

            # Periksa apakah yt-dlp mengembalikan kode error (returncode bukan 0)
            if result.returncode != 0:
                # Ambil pesan error spesifik dari stderr
                error_message = result.stderr.strip()
                # Lemparkan error yang berisi pesan dari yt-dlp
                raise RuntimeError(f"yt-dlp GAGAL. Pesan error dari YouTube: '{error_message}'")

            input_path = DEFAULT_INPUT_VIDEO_NAME
            print(f"--- Video berhasil diunduh sebagai {input_path} ---")
            is_video = True
        elif uploaded_file is not None:
            input_path = uploaded_file.name
            is_video   = input_path.lower().endswith(('.mp4','.mov','.avi','.mkv'))
        else:
            raise gr.Error("Harap isi URL YouTube **atau** unggah file.")
    except Exception as e:
        log_and_raise("mengunduh/menentukan input", e)

    # 1. Ekstrak audio
    try:
        progress(0.10, desc="Ekstrak audio…")
        audio_path = preprocess_media(input_path, TEMP_AUDIO_PATH)
    except Exception as e:
        log_and_raise("ekstrak audio", e)

    # 2. Transkripsi
    try:
        progress(0.30, desc="Transkripsi…")
        segments = transcribe_audio(audio_path)
    except Exception as e:
        log_and_raise("transkripsi", e)

    # 3. Diarisasi
    try:
        progress(0.55, desc="Diarisasi…")
        turns = diarize_speakers(audio_path)
    except Exception as e:
        log_and_raise("diarisasi", e)

    # 4. Subtitle & dialog list
    try:
        progress(0.75, desc="Generate subtitle…")
        full_text = generate_final_transcript_and_srt(segments, turns, SUBTITLE_PATH)
        dialog    = srt_to_dialog_list(SUBTITLE_PATH)
    except Exception as e:
        log_and_raise("generate subtitle", e)

    # 5. Hardsub (opsional)
    video_out = None
    if is_video:
        try:
            progress(0.85, desc="Render hardsub…")
            burn_subtitles_into_video(input_path, SUBTITLE_PATH, OUTPUT_VIDEO_PATH)
            video_out = OUTPUT_VIDEO_PATH
        except Exception as e:
            log_and_raise("render hardsub", e)

    # 6. Ringkasan
    # try:
    #     progress(0.95, desc="Ringkasan…")
    #     summary = summarizer(full_text[:4096], max_length=150, min_length=50, do_sample=False)[0]["summary_text"]
    # except Exception as e:
    #     log_and_raise("ringkasan", e)

    # 6. Ringkasan (MODIFIED)
    try:
        progress(0.95, desc="Membuat ringkasan dengan FLAN-T5...")
        # Panggil fungsi peringkasan kustom Anda
        summary = summarize_with_flan_t5(full_text, summary_model, summary_tokenizer)
    except Exception as e:
        log_and_raise("ringkasan", e)

    progress(1, desc=f"Selesai ✓  ({time.perf_counter()-t0:.1f}s)")
    return (
        video_out,
        dialog_to_chat_tuples(dialog),
        summary,
        full_text
    )

# ------------  QA handler (dipanggil setelah pipeline selesai) -----------------
def answer_question(history, user_msg, full_text):
    if not full_text:
        raise gr.Error("Jalankan pipeline dulu.")
    result = qa_pipeline(question=user_msg, context=full_text)
    ans    = result["answer"] #if result["score"] > 0.1 else "Maaf, saya tidak menemukan jawaban relevan."
    history.append((user_msg, ans))
    return history, ""

# ------------------------  BUILD GRADIO UI  ------------------------------------
with gr.Blocks(title="Smart AV Analyzer") as demo:
    gr.Markdown("## 🎙️ Smart Audio/Video Analyzer")

    with gr.Row():
        youtube_in = gr.Textbox(label="YouTube URL (opsional)")
        file_in    = gr.File(label="Upload Video/Audio (opsional)")

    run_btn = gr.Button("🚀  Jalankan Analisis")

    with gr.Row():
        video_out = gr.Video(label="Video + Subtitle", visible=False)
        summary_out = gr.Textbox(label="Ringkasan", lines=8)

    transcript_html = gr.HTML(label="Transcript Chat", elem_id="transcript-box")
    qa_chat         = gr.Chatbot(label="Tanya Jawab Berdasarkan Video", height=300)
    qa_user_in      = gr.Textbox(label="Ketik pertanyaan lalu tekan Enter")

    hidden_context  = gr.State(value="")   # menyimpan full_text utk QA

    # --- klik RUN: jalankan pipeline & tampilkan hasil ---
    def _run_debug(youtube_url, file_obj):
        v, dialog_tuples, summary, context = run_full_pipeline(youtube_url, file_obj)
        html_view = dialog_to_html(
            [{"speaker": t[0], "content": t[1]} for t in dialog_tuples]
        )
        return (
            gr.update(value=v, visible=bool(v)),
            html_view,
            summary,
            context
        )

    run_btn.click(
        fn=_run_debug,
        inputs=[youtube_in, file_in],
        outputs=[video_out, transcript_html, summary_out, hidden_context]
    )

    # --- QA on Enter ---
    qa_user_in.submit(
        answer_question,
        inputs=[qa_chat, qa_user_in, hidden_context],
        outputs=[qa_chat, qa_user_in]
    )

demo.launch(share=True, debug=True)

Loading custom model from: /content/drive/MyDrive/flan-t5-dialogsum-finetuned
✅ Successfully loaded fine-tuned FLAN-T5 summarization model.


/tmp/ipython-input-5-3017621351.py:249: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  qa_chat         = gr.Chatbot(label="Tanya Jawab Berdasarkan Video", height=300)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8e4d1fbbe268b396dc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


--- Mencoba mengunduh dari YouTube: https://www.youtube.com/shorts/Q0BOH_s9gSU ---
--- Video berhasil diunduh sebagai input_video.mp4 ---
--- Memproses file media: input_video.mp4 ---
--- Audio berhasil diekstrak dan disimpan di temp_processing_audio.wav ---

--- Memulai Transkripsi ---
-> Bahasa terdeteksi: en (Probabilitas: 1.00)
--- Transkripsi Selesai ---

--- Memulai Diarisasi Pembicara ---
-> Diarisasi selesai. Ditemukan 1 pembicara.
--- Diarisasi Selesai ---

--- Hasil Akhir Transkrip dengan Pembicara ---
[SPEAKER_00] (00:00:00,000 --> 00:00:03,840): I don't actually know if Christian Bale did say this, but the quote is,
[SPEAKER_00] (00:00:03,840 --> 00:00:05,840): If you have a problem with me, text me.
[SPEAKER_00] (00:00:05,840 --> 00:00:09,280): And if you don't have my number, you don't know me well enough to have a problem with me.
[SPEAKER_00] (00:00:09,280 --> 00:00:10,320): That is so good.

--- File subtitle telah disimpan di: subtitles.srt ---

--- Menambahkan subtit